# **AlexNet Paper implementation**



*   Paper link : https://proceedings.neurips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf



In [ ]:
import torch
import torch.nn as nn

In [ ]:
class AlexNet(nn.Module):
  def __init__(self, classes=1000):
    """
    classes : number of classes to classify
    """
    super().__init__()
    self.classes = classes

    # conv layers
    self.net = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),
        nn.ReLU(),
        nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
        nn.MaxPool2d(kernel_size=3, stride=2),

        nn.Conv2d(96, 256, 5, padding=2),
        nn.ReLU(),
        nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
        nn.MaxPool2d(kernel_size=3, stride=2),

        nn.Conv2d(256, 384, 3, padding=1),
        nn.ReLU(),

        nn.Conv2d(384, 384, 3, padding=1),
        nn.ReLU(),

        nn.Conv2d(384, 256, 3, padding=1),
        nn.ReLU(),

        nn.MaxPool2d(kernel_size=3, stride=2),
    )

    # fully connected layers
    self.fc = nn.Sequential(
        nn.Dropout(p=0.5, inplace=True),

        nn.Linear(in_features=(256 * 6 * 6), out_features=4096),
        nn.ReLU(),
        nn.Dropout(p=0.5, inplace=True),

        nn.Linear(in_features=4096, out_features=4096),
        nn.ReLU(),

        nn.Linear(in_features=4096, out_features=classes),
    )

    self.initialize_bias_weights()

  # initialize bias and weights
  def initialize_bias_weights(self):
    for layer in self.net:
      if isinstance(layer, nn.Conv2d):
        nn.init.normal_(layer.weight, mean=0, std=0.01)
        nn.init.constant_(layer.bias, 0)

    for idx in [4, 10, 12]:
      nn.init.constant_(self.net[idx].bias, 1)

  def forward(self, x):
    x = self.net(x)
    x = x.view(-1, 256 * 6 * 6)
    x = self.fc(x)
    return x